<a href="https://colab.research.google.com/github/mubrij/HR_employee/blob/main/Financial_Inclusion_in_Africa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import requests
from tqdm.auto import tqdm

In [92]:
data_url = "https://api.zindi.africa/v1/competitions/fraud-detection-in-electricity-and-gas-consumption-challenge/files/train.zip"

In [93]:
token = {'auth_token': 'b'}
file_name = "train.zip"

In [94]:
def zindi_data_downloader(url, token, file_name):
    # Get the competition data
    competition_data = requests.post(url = data_url, data= token, stream=True)
    
    # Progress bar monitor download
    pbar = tqdm(desc=file_name, total=int(competition_data.headers.get('content-length', 0)), unit='B', unit_scale=True, unit_divisor=512)
    # Create and Write the data to colab drive in chunks
    handle = open(file_name, "wb")
    for chunk in competition_data.iter_content(chunk_size=512): # Download the data in chunks
        if chunk: # filter out keep-alive new chunks
                handle.write(chunk)
        pbar.update(len(chunk))
    handle.close()
    pbar.close()

In [95]:
zindi_data_downloader(url=data_url, token=token, file_name=file_name)

train.zip: 0.00B [00:00, ?B/s]

In [129]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

In [172]:
df = pd.read_csv("/content/Train (1).csv")
df.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [173]:
x_data =df["bank_account"]

In [174]:
train_data = pd.read_csv('/content/Train (1).csv')

# Drop the unique identifier columns
train_data.drop(['uniqueid', 'bank_account'], axis=1, inplace=True)

# Convert the string variables to categorical variables
train_data['country'] = train_data['country'].astype('category')
train_data['year'] = train_data['year'].astype('category')
train_data['cellphone_access'] = train_data['cellphone_access'].astype('category')
train_data['gender_of_respondent'] = train_data['gender_of_respondent'].astype('category')
train_data['relationship_with_head'] = train_data['relationship_with_head'].astype('category')
train_data['marital_status'] = train_data['marital_status'].astype('category')
train_data['education_level'] = train_data['education_level'].astype('category')
train_data['job_type'] = train_data['job_type'].astype('category')


train_data.replace('Unknown', np.nan, inplace=True)

train_data['household_size'].fillna(train_data['household_size'].median(), inplace=True)
train_data['age_of_respondent'].fillna(train_data['age_of_respondent'].median(), inplace=True)

train_data['log_household_size'] = np.log(train_data['household_size'])
train_data = pd.get_dummies(train_data, drop_first=True)

#numerical_vars = ['age_of_respondent', 'log_household_size']
#train_data[numerical_vars] = (train_data[numerical_vars] - train_data[numerical_vars].mean()) / train_data[numerical_vars].std()


In [175]:
test_data = pd.read_csv('/content/Test.csv')

submission_df = pd.DataFrame({'uniqueid': test_data['uniqueid'] + ' x ' + test_data['country'], 'bank_account': predictions.reshape(-1)})

test_id = test_data['uniqueid']

test_data.drop('uniqueid', axis=1, inplace=True)

test_data['country'] = test_data['country'].astype('category')
test_data['year'] = test_data['year'].astype('category')
test_data['cellphone_access'] = test_data['cellphone_access'].astype('category')
test_data['gender_of_respondent'] = test_data['gender_of_respondent'].astype('category')
test_data['relationship_with_head'] = test_data['relationship_with_head'].astype('category')
test_data['marital_status'] = test_data['marital_status'].astype('category')
test_data['education_level'] = test_data['education_level'].astype('category')
test_data['job_type'] = test_data['job_type'].astype('category')

test_data.replace('Unknown', np.nan, inplace=True)

test_data['household_size'].fillna(test_data['household_size'].median(), inplace=True)
test_data['age_of_respondent'].fillna(test_data['age_of_respondent'].median(), inplace=True)

test_data['log_household_size'] = np.log(test_data['household_size'])

test_data = pd.get_dummies(test_data, drop_first=True)

#numerical_vars = ['age_of_respondent', 'log_household_size']
#test_data[numerical_vars] = (test_data[numerical_vars] - test_data[numerical_vars].mean()) / test_data[numerical_vars].std()


In [176]:
test_df = pd.DataFrame(scale(test_data))
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,-0.351983,-0.510678,-0.077567,-0.768506,-0.625898,-0.313201,-0.625898,1.371156,1.260216,0.578190,...,-0.192113,-0.554215,7.730397,-0.214589,-0.10007,-0.55452,-0.167734,-0.209225,-0.351123,-0.615782
1,1.455835,0.780101,1.269161,-0.768506,-0.625898,-0.313201,-0.625898,1.371156,1.260216,0.578190,...,5.205258,-0.554215,-0.129359,4.660066,-0.10007,-0.55452,-0.167734,-0.209225,-0.351123,-0.615782
2,-0.351983,2.378208,-0.077567,-0.768506,-0.625898,-0.313201,-0.625898,1.371156,-0.793515,-1.729536,...,-0.192113,-0.554215,-0.129359,-0.214589,-0.10007,-0.55452,-0.167734,-0.209225,2.848001,-0.615782
3,1.003881,0.042513,1.024148,-0.768506,-0.625898,-0.313201,-0.625898,1.371156,-0.793515,-1.729536,...,-0.192113,-0.554215,-0.129359,-0.214589,-0.10007,-0.55452,-0.167734,-0.209225,2.848001,-0.615782
4,-0.351983,-1.371197,-0.077567,-0.768506,-0.625898,-0.313201,-0.625898,1.371156,1.260216,-1.729536,...,-0.192113,-0.554215,-0.129359,-0.214589,-0.10007,-0.55452,-0.167734,-0.209225,2.848001,-0.615782


In [177]:
df = pd.DataFrame(scale(train_data))

In [178]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,-0.358007,-0.896188,-0.084265,-0.768532,-0.625798,-0.313165,-0.625798,1.371009,-0.800064,0.589721,...,-0.187994,-0.548535,-0.129331,-0.216688,-0.103011,-0.558758,-0.16548,-0.219362,-0.346916,1.629263
1,0.539834,1.888279,0.726380,-0.768532,-0.625798,-0.313165,-0.625798,1.371009,-0.800064,-1.695716,...,-0.187994,-0.548535,-0.129331,-0.216688,9.707670,-0.558758,-0.16548,-0.219362,-0.346916,-0.613775
2,0.539834,-0.775124,0.726380,-0.768532,-0.625798,-0.313165,-0.625798,1.371009,1.249899,0.589721,...,5.319318,-0.548535,-0.129331,-0.216688,-0.103011,-0.558758,-0.16548,-0.219362,-0.346916,1.629263
3,0.539834,-0.290869,0.726380,-0.768532,-0.625798,-0.313165,-0.625798,1.371009,-0.800064,0.589721,...,-0.187994,-0.548535,-0.129331,4.614936,-0.103011,-0.558758,-0.16548,-0.219362,-0.346916,-0.613775
4,1.886596,-0.775124,1.472243,-0.768532,-0.625798,-0.313165,-0.625798,1.371009,1.249899,-1.695716,...,-0.187994,-0.548535,-0.129331,-0.216688,-0.103011,1.789683,-0.16548,-0.219362,-0.346916,-0.613775


In [180]:
y_df = x_data.map({'Yes': 1, 'No': 0})
y_df

0        1
1        0
2        1
3        0
4        0
        ..
23519    0
23520    0
23521    0
23522    0
23523    0
Name: bank_account, Length: 23524, dtype: int64

In [181]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df, y_df, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(350, input_shape=(34,), activation='relu'))
model.add(Dense(260, activation='relu'))
model.add(Dense(220, activation='sigmoid'))
model.add(Dense(260, activation=tf.keras.activations.softmax))
model.add(Dense(220, activation='relu'))
model.add(Dense(240, activation=tf.keras.activations.softmax))
model.add(Dense(240, activation='sigmoid'))
model.add(Dense(380, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=10, epochs=150, validation_data=(X_val, y_val))

Epoch 1/150
1882/1882 [==============================] - 22s 11ms/step - loss: 0.4130 - accuracy: 0.8578 - val_loss: 0.3989 - val_accuracy: 0.8635
Epoch 2/150
1882/1882 [==============================] - 18s 9ms/step - loss: 0.4112 - accuracy: 0.8581 - val_loss: 0.4022 - val_accuracy: 0.8635
Epoch 3/150
1882/1882 [==============================] - 21s 11ms/step - loss: 0.4106 - accuracy: 0.8581 - val_loss: 0.4037 - val_accuracy: 0.8635
Epoch 4/150
1882/1882 [==============================] - 18s 9ms/step - loss: 0.4097 - accuracy: 0.8581 - val_loss: 0.3996 - val_accuracy: 0.8635
Epoch 5/150
1882/1882 [==============================] - 20s 10ms/step - loss: 0.4101 - accuracy: 0.8581 - val_loss: 0.4026 - val_accuracy: 0.8635
Epoch 6/150
1882/1882 [==============================] - 18s 9ms/step - loss: 0.4097 - accuracy: 0.8581 - val_loss: 0.4006 - val_accuracy: 0.8635
Epoch 7/150
1882/1882 [==============================] - 18s 10ms/step - loss: 0.4100 - accuracy: 0.8581 - val_loss: 0.40

In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
submission_df.to_csv('Financial_Inclusion_in Africa_submission3.csv', index=False)

In [ ]:
tf.saved_model.save(model, 'model.h5')

In [ ]:
test_data